In [27]:
import pandas as pd
from IPython.display import display

data_bestand = "../data/ion.csv"
data_ion = pd.read_csv(data_bestand)

df = data_ion.copy()
df['Class'] = df['Class'].map({'good': 0, 'bad': 1})

df_train = df.sample(frac=0.7, random_state=0)
df_valid = df.drop(df_train.index)

max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)

df_train = (df_train - min_) / (max_ - min_)
df_test = (df_valid - min_) / (max_ - min_)
df_train.dropna(axis=1, inplace=True) # Wis alle lege eigenschappen in de 2 kolommen
df_test.dropna(axis=1, inplace=True)

X_train = df_train.drop('Class', axis=1)
X_test = df_test.drop('Class', axis=1)
y_train = df_train['Class']
y_test = df_test['Class']

X_train

,Number,V1,V3,V4,V5,V6,V7,V8,V9,V10,...,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34
6,0.014327,1.0,0.987940,0.44699,0.973005,0.396000,0.964030,0.358250,0.929980,0.363290,...,0.613960,0.091830,0.568295,0.087450,0.523030,0.088025,0.478690,0.09341,0.430840,0.095125
52,0.146132,1.0,1.000000,1.00000,1.000000,1.000000,0.955050,1.000000,0.365150,1.000000,...,0.950070,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.32657,1.000000,0.674225
114,0.323782,1.0,0.956765,0.90793,0.135135,1.000000,0.302670,0.778675,0.527025,0.648650,...,0.499985,0.500010,0.499995,0.500000,0.500000,0.500000,0.500000,0.50000,0.500000,0.500000
45,0.126074,0.0,0.500000,0.50000,0.500000,0.500000,0.500000,0.500000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.50000,0.500000,0.500000
106,0.300860,1.0,0.500000,0.50000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.500000,0.500000,1.000000,0.554880,0.500000,0.500000,0.500000,0.50000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.163324,1.0,1.000000,0.52906,0.972625,0.537090,0.999760,0.566155,1.000000,0.490445,...,0.958335,0.612155,0.942115,0.617855,0.942840,0.612555,0.891620,0.64788,0.917870,0.655830
41,0.114613,1.0,0.000000,0.00000,1.000000,1.000000,1.000000,0.428125,0.500000,0.500000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.00000,0.500000,0.500000
14,0.037249,1.0,1.000000,0.53690,1.000000,0.517100,1.000000,0.472185,1.000000,0.543820,...,1.000000,0.600165,1.000000,0.683715,0.978015,0.743205,1.000000,0.66246,1.000000,0.733560
155,0.441261,1.0,0.862070,0.49458,0.898520,0.505420,0.900000,0.500985,0.895075,0.505420,...,0.863055,0.492610,0.890205,0.503060,0.870445,0.474875,0.914780,0.51478,0.895075,0.503940


In [28]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(4, activation='relu', input_shape=[33]),
    layers.Dense(4, activation='relu'),    
    layers.Dense(1, activation='sigmoid'),
])

c:\Users\bthon\Documents\AI\Les_AI\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [31]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=0, # We verbergen de uitvoer omdat we teveel epochs hebben
)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_6" is incompatible with the layer: expected axis -1 of input shape to have value 33, but received input with shape (None, 34)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 34), dtype=float32)
  • training=True
  • mask=None

In [32]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[5:, ['loss', 'val_loss']].plot()
history_df.loc[5:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Beste validatieverlies: {:0.4f}" +\
      "\nBeste controle accuraatheiD: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_binary_accuracy'].max()))

NameError: name 'history' is not defined